In [2]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [170]:
df = pd.read_csv('QQQ_norm_2005-01-03 to 2020-06-18.csv')
df = df[27:]

In [180]:
len(df)

3865

In [169]:
df = pd.read_csv('QQQ_2005-02-08_to_2020-05-29.csv')

def format_data(df,start, end):
    training_data = df[start:end].drop('Date', axis = 1)
    return(training_data)
        

def create_xy(training_data, samples, forecast):
    x = []
    y = []
    L = len(training_data)
    samples = 100
    forecast = 20

    for i in range(samples, L - forecast):
        x.append(np.array(training_data[i-samples:i]))
        y.append(np.array(training_data[i+1:i+1+forecast]['pct chage price']))
    x = np.array(x)
    y = np.array(y)
    
    return([x,y])
#training_data is from QQQ.train

In [255]:
x_train = create_xy(training_data,100, 20)[0]
y_train = create_xy(training_data, 100, 20)[1]

In [260]:
y_train.shape

(2880, 20)

In [76]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [261]:
hyparam ={'loss':'mean_squared_error',
         'opt':'Adam',
         'epochs': 10,
         'batch_size':32}

regressor = Sequential()

regressor.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 120, activation = 'relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=y_train.shape[1]))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(x_train, y_train, epochs = 10, batch_size = 32)

Train on 2880 samples
Epoch 1/10
2880/2880 [==============================] - 55s 19ms/sample - loss: 1.7497e-04
Epoch 2/10
2880/2880 [==============================] - 24s 8ms/sample - loss: 1.7430e-04
Epoch 3/10
2880/2880 [==============================] - 23s 8ms/sample - loss: 1.7412e-04
Epoch 4/10
2880/2880 [==============================] - 23s 8ms/sample - loss: 1.7424e-04
Epoch 5/10
2880/2880 [==============================] - 23s 8ms/sample - loss: 1.7433e-04
Epoch 6/10
2880/2880 [==============================] - 24s 8ms/sample - loss: 1.7439e-04
Epoch 7/10
2880/2880 [==============================] - 23s 8ms/sample - loss: 1.7409e-04
Epoch 8/10
2880/2880 [==============================] - 24s 8ms/sample - loss: 1.7437e-04
Epoch 9/10
2880/2880 [==============================] - 23s 8ms/sample - loss: 1.7414e-04
Epoch 10/10
2880/2880 [==============================] - 23s 8ms/sample - loss: 1.7414e-04


In [ ]:
regressor.save('2020-06-21-2100_LSTM-100_to_20_prenorm.model')

In [309]:
test_df = pd.read_csv('QQQ_norm_2005-01-03 to 2020-06-18.csv',index_col = 0)

In [314]:
test_df

,pct chage price,vol change pct,12 EMA diff,26 EMA diff,norm RSI 9
Date,,,,,
2005-01-03,0.000000,0.000000,0.000000,0.000000,0.000000
2005-01-04,-0.018228,0.353095,0.000000,0.000000,0.000000
2005-01-05,-0.006189,-0.063662,0.000000,0.000000,0.000000
2005-01-06,-0.004930,-0.195355,0.000000,0.000000,0.000000
2005-01-07,0.005215,0.195944,0.000000,0.000000,0.000000
...,...,...,...,...,...
2020-06-12,0.007948,-0.013930,-0.001798,0.011310,0.530806
2020-06-15,0.012167,-0.369887,0.004525,0.016509,0.550584
2020-06-16,0.017173,0.079075,0.012339,0.023777,0.590909


In [313]:
training_data

,pct chage price,vol change pct,12 EMA diff,26 EMA diff,norm RSI 9
27,0.001078,0.183287,-0.004276,-0.008470,0.545817
28,0.014805,-0.015302,0.003727,-0.000499,0.558139
29,0.004509,-0.494784,0.005401,0.001787,0.566540
30,0.006602,1.393173,0.007856,0.004945,0.595744
31,-0.003673,-0.389118,0.004800,0.002737,0.664031
...,...,...,...,...,...
3022,0.005438,-0.114692,0.002313,0.004368,0.465251
3023,0.005658,0.045329,0.004777,0.006864,0.535529
3024,0.008770,0.166148,0.008405,0.010717,0.639445
3025,0.003281,-0.214548,0.008740,0.011553,0.656388


In [315]:
x_test = pd.read_csv('QQQ_train.csv', index_col = 0)

In [324]:
x_test = x_test[-100:]

In [325]:
x_test = np.array(x_test)

In [328]:
x_test = x_test.reshape(1, x_test.shape[0], x_test.shape[1])

In [329]:
y_pred = regressor.predict(x_test)

In [331]:
y_pred.shape

(1, 20)

In [332]:
preds = list(y_pred[0])

In [333]:
preds

[-0.0010228489,
 -0.002060439,
 0.00060101255,
 0.0011777305,
 -0.00056102744,
 -0.00040712566,
 -0.0004621009,
 0.00022253365,
 0.00057223975,
 0.0008967547,
 0.00053915416,
 0.0012162081,
 -5.0692717e-05,
 0.0005116693,
 0.0008288241,
 8.6017695e-05,
 0.00083469803,
 0.00011076078,
 -0.00080830295,
 0.000530856]